In [1]:
import json
import torch
import torch.nn as nn
import random
from tqdm import tqdm
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
fact_checking_model = GPT2LMHeadModel.from_pretrained('gpt2')
fact_checking_model.cuda()
checkpoint = torch.load('save_fever2')
fact_checking_model.load_state_dict(checkpoint['model_state_dict'])
_ = fact_checking_model.eval()

In [4]:
def get_text_up_to_question(text):
    _claim_yn = 'The evidence supports the claim:\n'
    return text[:text.find(_claim_yn) + len(_claim_yn)]

In [5]:
def get_answer_from_text(text):
    _claim_yn = 'The evidence supports the claim:\n'
    pos = text.find(_claim_yn) + len(_claim_yn)
    return text[pos]

In [6]:
def generate_answer(model, text):
    prompt = get_text_up_to_question(text)
    tokens = tokenizer.encode(prompt, return_tensors='pt')
    _length = 1
    tokens_length = tokens.shape[1]
    if tokens_length + _length >= 1024:
        raise RuntimeError('Text is longer than 1024')
    output = model.generate(
             tokens.cuda(),
             max_length=tokens_length + _length, 
             pad_token_id=50256
    )
    to_return = tokenizer.decode(output[0], skip_special_tokens=True)
    perplexity = float(model(output, labels=output)[0])
    return get_answer_from_text(to_return), perplexity

# Question Answering part

In [7]:
_question_prompt = '\nQ: '
_answer_prompt = '\nA: '
    
def get_text_up_to_question_number(text, number):
    pos = text.find(_answer_prompt)
    for _ in range(number):
        pos = text.find(_answer_prompt, pos + 1)
    return text[0:pos + 1]
    
def get_answers_number(text):
    return text.count(_answer_prompt)

def get_answer_number(text, number):
    pos = text.find(_answer_prompt)
    for _ in range(number):
        pos = text.find(_answer_prompt, pos + 1)
    end = text.find('\n', pos + len(_answer_prompt))
    return text[pos + len(_answer_prompt):end]

def get_question_number(text, number):
    pos = text.find(_question_prompt)
    for _ in range(number):
        pos = text.find(_question_prompt, pos + 1)
    end = text.find('\n', pos + len(_question_prompt))
    return text[pos + len(_question_prompt):end]

def get_all_answers(dev_dict, dev_index):
    answers = [[item['input_text'] for item in dev_dict['data'][dev_index]['answers']]]
    return [list(set([answers[j][i] for j in range(len(answers))])) for i in range(len(answers[0]))]

In [8]:
def get_text_up_to_discussion(item):
    text = 'In the text below two people are discussing a story.\n\n'
    text += 'Story:\n' + item['story'] + '\n\n'
    text += 'Discussion:\n'
    return text

In [9]:
def get_text_from_data_item(item, max_num_questions=0, question_number=-1, last_question=True):
    text = 'In the text below two people are discussing a story.\n\n'
    text += 'Story:\n' + item['story'] + '\n\n'
    text += 'Discussion:\n'
    text += '\n'.join(['Q: ' + q['input_text'] 
                       + '\nA: ' + a['input_text'] 
                       for q, a in zip(item['questions'][max(0,question_number-max_num_questions):question_number+1], 
                                       item['answers'][max(0,question_number-max_num_questions):question_number+1]) 
                      ])
    if not last_question:
        text = '\n'.join(text.split('\n')[:-1]) + '\n'
    return text

In [10]:
train_dict = json.load(open('../data/coqa-train-v1.0.json', encoding='utf8'))
train_list = json.load(open('../data/qa_train_list.json', encoding='utf8'))
dev_dict = json.load(open('../data/coqa-dev-v1.0.json', encoding='utf8'))
dev_list = json.load(open('../data/qa_dev_list.json', encoding='utf8'))

In [11]:
def get_text_from_data_item(item, max_num_questions=0, question_number=-1, last_question=True):
    text = 'In the text below two people are discussing a story.\n\n'
    text += 'Story:\n' + item['story'] + '\n\n'
    text += 'Discussion:\n'
    text += '\n'.join(['Q: ' + q['input_text'] 
                       + '\nA: ' + a['input_text'] 
                       for q, a in zip(item['questions'][max(0,question_number-max_num_questions):question_number+1], 
                                       item['answers'][max(0,question_number-max_num_questions):question_number+1]) 
                      ])
    if not last_question:
        text = '\n'.join(text.split('\n')[:-1]) + '\n'
    return text

In [12]:
def get_description_from_data_item(item):
    return item['story']

def get_dialogue_from_data_item(item, max_num_questions=0, question_number=-1, last_question=True):
    text = ''
    text += ' '.join([q['input_text'] + ' ' + a['input_text'] + '.'
                       for q, a in zip(item['questions'][max(0,question_number-max_num_questions):question_number+1], 
                                       item['answers'][max(0,question_number-max_num_questions):question_number+1]) 
                      ])
    if not last_question:
        text = '?'.join(text.split('?')[:-1]) + '?'
    return text

In [13]:
def create_claim_from_description_and_dialogue(description, dialogue):
    if dialogue[-1] == '.':
        dialogue = dialogue[:-1]
    text = 'Evidence:\n'
    text += description.replace('\n\n', '\n') + '\n\n'
    text += 'Claim:\n'
    text += dialogue + '\n\n'
    text += 'The evidence supports the claim:\n'
    return text

In [14]:
def get_positive_list_of_claims():
    claims = []
    
    total_number_of_questions = 0
    correct_answers = 0
    wrong_predictions = []
    false_positives = []
    dlist = train_list
    for index, text in tqdm(enumerate(dlist), total=len(dlist)):
        total_questions = get_answers_number(text)
        all_answers = get_all_answers(train_dict, index)
        for number in range(total_questions):
            small_text = get_text_from_data_item(train_dict['data'][index], 
                                                 max_num_questions=5,
                                                 question_number=number,
                                                 last_question=False)
            description = get_description_from_data_item(train_dict['data'][index])
            dialogue = get_dialogue_from_data_item(train_dict['data'][index],
                                       max_num_questions=5, 
                                       question_number=number,
                                       last_question=True)
            
            last_answer = dialogue.split('?')[-1].strip().lower()
            if stripped_answer(last_answer) not in ['yes', 'no']:
                continue
        
            claims.append(create_claim_from_description_and_dialogue(description, dialogue) + 'Y')
    return claims

In [15]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.cuda()
checkpoint = torch.load('save_small' + str(0))
model.load_state_dict(checkpoint['model_state_dict'])
_ = model.eval()

In [16]:
def generate_multiple_answers(model, prompt, num_replicas=2):
    tokens = tokenizer.encode(prompt, return_tensors='pt')
    _length = 50
    tokens_length = tokens.shape[1]
    if tokens_length + _length > 1024:
        return ''
    
    outputs = []
    for _ in range(num_replicas):
        output = model.generate(
             tokens.cuda(),
             max_length=tokens_length + _length,
             pad_token_id=50256
        )
        output = tokenizer.decode(output[0], skip_special_tokens=True)
        offset = len(prompt)
        start = offset + 1
        end = output.find('\n', start)
        outputs.append(output[start:end].split(':')[-1].strip())
        
    return list(set(outputs))

In [17]:
def stripped_answer(text):
    return text.lower().strip().replace('.', '')

In [18]:
import numpy as np
from sentence_transformers import SentenceTransformer

device = 'cuda'

sentence_model = SentenceTransformer('msmarco-distilbert-base-v3')
sentence_model = sentence_model.to(device)

In [19]:
def get_embeddings_from_text(text):
    outputs = sentence_model.encode(text)
    return outputs


In [20]:
def answers_are_consistent(lhs, rhs):
                        
    label_embeddings = get_embeddings_from_text(lhs)
    prediction_embeddings = get_embeddings_from_text(rhs)
    if np.dot(label_embeddings, prediction_embeddings) \
       / np.linalg.norm(label_embeddings) \
       / np.linalg.norm(prediction_embeddings) \
                       > 0.3:
        return True

    return False

In [21]:
def answers_are_consistent_original(lhs, rhs):
    prediction = stripped_answer(lhs)
    label = stripped_answer(rhs)
    if prediction.lower() == label.lower():
        return True
    elif prediction.lower() in label.lower():
        return True
    elif label.lower() in prediction.lower():
        return True
    return False

In [22]:
import random
random.seed(42)

def get_adversarial_list_of_texts(text_list, text_dict):
    texts = []
    
    total_number_of_questions = 0
    correct_answers = 0
    wrong_predictions = []

    false_positives = []
    dlist = text_list
    random.shuffle(text_list)
    for index, text in tqdm(enumerate(dlist), total=len(dlist)):
        total_questions = get_answers_number(text)
        all_answers = get_all_answers(text_dict, index)
        substitution_pair_list = []
        discussion = ''
        new_text = get_text_up_to_discussion(text_dict['data'][index])
        for number in range(total_questions):
            small_text = get_text_from_data_item(text_dict['data'][index],
                                                 max_num_questions=5,
                                                 question_number=number,
                                                 last_question=False)
            dialogue = get_dialogue_from_data_item(text_dict['data'][index],
                           max_num_questions=5, 
                           question_number=number,
                           last_question=True)
            last_answer = dialogue.split('?')[-1].strip().lower()
            dialogue = '?'.join(dialogue.split('?')[:-1]) + '?'
            wrong_answer = ''
            if stripped_answer(last_answer) in ['yes', 'no', 'unknown']:
                continue

            else:
                generated_answers = generate_multiple_answers(model, small_text)
                for candidate in generated_answers:
                    if not answers_are_consistent(candidate, last_answer):
                        wrong_answer = candidate
                        break

            if not wrong_answer:
                continue
                
            if stripped_answer(wrong_answer) in ['yes', 'no', 'unknown']:
                continue
            
            if last_answer.replace('.', '') not in new_text:
                continue

            try:
                substitution_pair_list.append((last_answer, wrong_answer))
                discussion += 'Q: ' + text_dict['data'][index]['questions'][number]['input_text'] + '\n'
                discussion += 'WA: ' + wrong_answer + '\n'
                discussion += 'CA: ' + last_answer + '\n'
            except:
                print(f'index {index}, number {number} not in dictionary.')
                continue
        
        if not substitution_pair_list:
            continue
        
        #for correct_answer, wrong_answer in substitution_pair_list:
        #    new_text = new_text.replace(correct_answer.replace('.', ''), wrong_answer)
        if discussion:
            texts.append(new_text + discussion)
    return texts

In [23]:
adversarial_claims_train = get_adversarial_list_of_texts(train_list, train_dict)

  0%|          | 2/7199 [00:15<15:16:55,  7.64s/it]

index 1, number 10 not in dictionary.


  0%|          | 8/7199 [00:59<16:05:18,  8.05s/it]

index 8, number 15 not in dictionary.
index 8, number 16 not in dictionary.
index 8, number 17 not in dictionary.
index 8, number 18 not in dictionary.
index 8, number 19 not in dictionary.


  0%|          | 9/7199 [01:13<19:47:32,  9.91s/it]

...

index 7195, number 17 not in dictionary.


100%|█████████▉| 7198/7199 [17:18:34<00:09,  9.01s/it]

index 7198, number 18 not in dictionary.


100%|██████████| 7199/7199 [17:18:46<00:00,  8.66s/it]

index 7198, number 19 not in dictionary.


In [24]:
adversarial_claims_dev = get_adversarial_list_of_texts(dev_list, dev_dict)

  1%|          | 6/500 [00:47<1:00:37,  7.36s/it]

index 6, number 17 not in dictionary.
index 6, number 18 not in dictionary.


  1%|▏         | 7/500 [00:59<1:13:16,  8.92s/it]

index 6, number 19 not in dictionary.


  2%|▏         | 8/500 [01:13<1:26:54, 10.60s/it]

...

 99%|█████████▉| 496/500 [1:14:51<00:35,  8.78s/it]

index 496, number 10 not in dictionary.
index 496, number 11 not in dictionary.


 99%|█████████▉| 497/500 [1:14:58<00:25,  8.41s/it]

index 496, number 12 not in dictionary.


100%|██████████| 500/500 [1:15:20<00:00,  9.04s/it]


In [25]:
print(adversarial_claims_dev[3])

In the text below two people are discussing a story.

Story:
Kendra and Quinton travel to and from school every day. Kendra lives further from the bus stop than Quinton does, stops every morning at Quinton's house to join him to walk to the bus stop. Every afternoon, after school, when walking home from the bus stop they go in for cookies and milk that Quinton's mother has ready and waiting for them. Quinton can't eat cheese or cake so they had the same snack every day. They both work together on their homework and when they are done they play together. Kendra always makes sure to leave in time to get home for dinner. She doesn't want to miss story time which was right before bedtime. 

One morning Kendra walked up to Quinton's house, she thought something might be wrong because normally Quinton was waiting outside for her and on this morning he was not to be found. Kendra went up to the door and knocked. She waited and waited and yet no one answered. She saw that Quinton's mother's ca

In [26]:
json.dump(adversarial_claims_train, open('../data/adversarial_claims_train.json', 'w'))
json.dump(adversarial_claims_dev, open('../data/adversarial_claims_dev.json', 'w'))